# PostgreSQL Summary Stats and Window Functions
Here you can access the `summer_medals` table used in the course. To access the table, you will need to specify the `medals` schema in your queries (e.g., `medals.summer_medals`).

--- 
_Note: When using sample integrations such as those that contain course data, you have read-only access. You can run queries, but cannot make any changes such as adding, deleting, or modifying the data (e.g., creating tables, views, etc.)._

## Take Notes

Add notes about the concepts you've learned and SQL cells with queries you want to keep.

_Add your notes here_

In [ ]:
-- Add your own queries here
SELECT *
FROM medals.summer_medals
LIMIT 5

## Explore Datasets
Use the `summer_medals` table to explore the data and practice your skills!
- Select the `athlete`, `event`, and `year` from the `summer_medals` table.
    - Add another column, `previous_winner`, which contains the previous winner of the same event.
    - Filter your results for gold medalists.
- Return the `year`, total number of medalists per year, and running total number of medalists in the history of the Summer Olympics.
    - Order your results by year in ascending order.
- Return the `country`, `year`, and the number of gold medals earned.
   - Limit your results to the years 2004, 2008, and 2012.
   - Each country should have a subtotal of all gold medals earned across the three years.

## Transforming a table ordered by by rank into a 1D list 
`STRING_AGG(Column, 'separator')`

In [ ]:
WITH Country_Medals AS (
  SELECT
    Country,
    COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE Year = 2000
    AND Medal = 'Gold'
  GROUP BY Country),

  Country_Ranks AS (
  SELECT
    Country,
    RANK() OVER (ORDER BY Medals DESC) AS Rank
  FROM Country_Medals
  ORDER BY Rank ASC)

-- Compress the countries column
SELECT STRING_AGG(Country, ', ')
FROM Country_Ranks
-- Select only the top three ranks
WHERE Rank <= 3;

## Exploring all possible groupings while mantaining hierarchy:
`ROLLUP(Column1,...)`

## Renaming null to something more informative: 
`COALESCE(Column, 'Informative group name')`

In [4]:
SELECT
  -- Replace the nulls in the columns with meaningful text
  COALESCE(Country, 'All countries') AS Country,
  COALESCE(Gender, 'All genders') AS Gender,
  COUNT(*) AS Awards
FROM medals.Summer_Medals
WHERE
  Year = 2004
  AND Medal = 'Gold'
  AND Country IN ('DEN', 'NOR', 'SWE')
GROUP BY ROLLUP(Country, Gender)
ORDER BY Country ASC, Gender ASC;

,country,gender,awards
0,All countries,All genders,29
1,DEN,All genders,19
2,DEN,Men,4
3,DEN,Women,15
4,NOR,All genders,5
5,NOR,Men,3
6,NOR,Women,2
7,SWE,All genders,5
8,SWE,Men,4
9,SWE,Women,1


## Exploring all possible groupings, hierarchy flows in all directions: 
`CUBE(Column1,...)`

In [6]:
-- Count the medals per gender and medal type
SELECT
  COALESCE(Gender, 'All genders') AS Gender,
  COALESCE(Medal, 'All medals') AS Medal,
  Count(*) AS Awards
FROM medals.Summer_Medals
WHERE
  Year = 2012
  AND Country = 'RUS'
-- Get all possible group-level subtotals
GROUP BY CUBE(Gender, Medal)
ORDER BY Gender ASC, Medal ASC;

,gender,medal,awards
0,All genders,All medals,130
1,All genders,Bronze,51
2,All genders,Gold,47
3,All genders,Silver,32
4,Men,All medals,64
5,Men,Bronze,34
6,Men,Gold,23
7,Men,Silver,7
8,Women,All medals,66
9,Women,Bronze,17


## Pivot: advanced example that uses `Rank`

In [ ]:
CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
  WITH Country_Awards AS (
    SELECT
      Country,
      Year,
      COUNT(*) AS Awards
    FROM Summer_Medals
    WHERE
      Country IN ('FRA', 'GBR', 'GER')
      AND Year IN (2004, 2008, 2012)
      AND Medal = 'Gold'
    GROUP BY Country, Year)

  SELECT
    Country,
    Year,
    RANK() OVER
      (PARTITION BY Year
       ORDER BY Awards DESC) :: INTEGER AS rank
  FROM Country_Awards
  ORDER BY Country ASC, Year ASC;
-- Fill in the correct column names for the pivoted table
$$) AS ct ("Country" VARCHAR,
           "2004" INTEGER,
           "2008" INTEGER,
           "2012" INTEGER)


**Output:**
| Country | 2004 | 2008 | 2012 |
|---------|------|------|------|
| FRA     | 2  | 3  | 3  |
| GBR     | 3  | 2  | 1  |
| GER     | 1  | 1  | 2  |


## Pivot: simple example


In [ ]:
-- Create the correct extension to enable CROSSTAB
CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
  SELECT
    Gender, Year, Country
  FROM Summer_Medals
  WHERE
    Year IN (2008, 2012)
    AND Medal = 'Gold'
    AND Event = 'Pole Vault'
  ORDER By Gender ASC, Year ASC;
-- Fill in the correct column names for the pivoted table
$$) AS ct (Gender VARCHAR,
           "2008" VARCHAR,
           "2012" VARCHAR)

ORDER BY Gender ASC;

**Instead of:**
| Gender | Year | Country |
|--------|------|---------|
| Men    | 2008 | AUS     |
| Men    | 2012 | FRA     |
| Women  | 2008 | RUS     |
| Women  | 2012 | USA     |

**A more readable output:**

| Gender | 2008 | 2012 |
|--------|------|------|
| Men    | AUS  | FRA  |
| Women  | RUS  | USA  |


## Using Windows to create new columns for easy comparison 

The course included: 
- `LAST_VALUE`, `FIRST_VALUE`, `LEAD`
- `PAGING`
- `RANK` to create a new column with a numerical order
- `DENSE RANK` gives a different rank even though it is a tie

## Window Functions with subclauses: `Order by `and `Partition by`

In [2]:
WITH Athletics_Gold AS (
  SELECT DISTINCT
    Gender, Year, Event, Country
  FROM medals.Summer_Medals
  WHERE
    Year >= 2000 AND
    Discipline = 'Athletics' AND
    Event IN ('100M', '10000M') AND
    Medal = 'Gold')

SELECT
  Gender, Year, Event,
  Country AS Champion,
  -- Fetch the previous year's champion by gender and event
  LAG(Country) OVER (PARTITION BY gender, event
            ORDER BY Year ASC) AS Last_Champion
FROM Athletics_Gold
ORDER BY Event ASC, Gender ASC, Year ASC;

,gender,year,event,champion,last_champion
0,Men,2000,10000M,ETH,None
1,Men,2004,10000M,ETH,ETH
2,Men,2008,10000M,ETH,ETH
3,Men,2012,10000M,GBR,ETH
4,Women,2000,10000M,ETH,None
5,Women,2004,10000M,CHN,ETH
6,Women,2008,10000M,ETH,CHN
7,Women,2012,10000M,ETH,ETH
8,Men,2000,100M,USA,None
9,Men,2004,100M,USA,USA


## Window Functions: use `LAG` to see the outputs from different windows

In [1]:
WITH Weightlifting_Gold AS (
  SELECT
    -- Return each year's champions' countries
    Year,
    Country AS champion
  FROM medals.Summer_Medals
  WHERE
    Discipline = 'Weightlifting' AND
    Event = '69KG' AND
    Gender = 'Men' AND
    Medal = 'Gold')

SELECT
  Year, Champion,
  -- Fetch the previous year's champion
  LAG(Champion) OVER
    (ORDER BY Year ASC) AS Last_Champion
FROM Weightlifting_Gold
ORDER BY Year ASC;

,year,champion,last_champion
0,2000,BUL,None
1,2004,CHN,BUL
2,2008,CHN,CHN
3,2012,CHN,CHN


## Window Function basics: using a moving `SUM`


In [2]:
WITH Country_Medals AS (
  SELECT
    Year, Country, COUNT(*) AS Medals
  FROM medals.Summer_Medals
  GROUP BY Year, Country)

SELECT
  Year, Country, Medals,
  -- Calculate each country's 3-game moving total
  SUM(Medals) OVER
    (PARTITION BY Country
     ORDER BY Year ASC
     ROWS BETWEEN
     2 PRECEDING AND CURRENT ROW) AS Medals_MS
FROM Country_Medals
ORDER BY Country ASC, Year ASC;

,year,country,medals,medals_ms
0,2012,,4,4
1,2008,AFG,1,1
2,2012,AFG,1,2
3,1988,AHO,1,1
4,1984,ALG,2,2
...,...,...,...,...
1153,2004,ZIM,3,19
1154,2008,ZIM,4,23
1155,1896,ZZX,6,6
1156,1900,ZZX,34,40


## Window Functions for summarizing outputs, creating new groupings and more

Unlike `GROUP BY` results, window functions don't reduce the number of rows in the output. Which lets use delve deep into the results and explore different windows (groupings)